In [1]:
import requests
import xmltodict
import lxml
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re 

Экспортируйте с OSM часть карты, на которой находитесь вы (или любую другую интересную вам). Посчитайте на карте объекты какого-либо типа.

In [ ]:
# Скачаем карту района по request url'у
# И найдем количество школ в этом районе

xml = urlopen("https://www.openstreetmap.org/api/0.6/map?bbox=30.1955%2C60.003%2C30.2464%2C60.0081").read().decode('utf-8')
soup1 = BeautifulSoup(xml, 'lxml').find_all('node')
schools_in_nodes = BeautifulSoup(str(soup1), 'lxml').find_all('tag', k='amenity', v='school')

soup2 = BeautifulSoup(xml, 'lxml').find_all('way')
schools_in_ways = BeautifulSoup(str(soup2), 'lxml').find_all('tag', k='amenity', v='school')

result = schools_in_nodes + schools_in_ways
print(len(result))


In [ ]:
xml = urlopen("https://www.openstreetmap.org/api/0.6/map?bbox=30.1955%2C60.003%2C30.2464%2C60.0081").read().decode('utf-8')

parsedxml = xmltodict.parse(xml)

In [ ]:
names = []
schools = 0
for node in parsedxml['osm']['node']:
    flag = 0
    if 'tag' in node:
        tags = node['tag']
        if isinstance(tags, list):
            for tag in tags:
                if '@k' in tag and tag['@k'] == 'amenity' and tag['@v'] == 'school':
                    flag = 1
                    schools += 1
                if '@k' in tag and tag['@k'] == 'name':
                    name = tag['@v']
            if flag:
                names.append(name)

print(schools)
print(names)

In [ ]:
for node in parsedxml['osm']['way']:
    flag = 0
    if 'tag' in node:
        tags = node['tag']
        if isinstance(tags, list):
            for tag in tags:
                if '@k' in tag and tag['@k'] == 'amenity' and tag['@v'] == 'school':
                    flag = 1
                    schools += 1
                if '@k' in tag and tag['@k'] == 'name':
                    name = tag['@v']
            if flag:
                names.append(name)

print(schools)
print(names)

Рассмотрим два HTML-документа A и B.
Из A можно перейти в B за один переход, если в A есть ссылка на B, т. е. внутри A есть тег 'a href="B">', возможно с дополнительными параметрами внутри тега.
Из A можно перейти в B за два перехода если существует такой документ C, что из A в C можно перейти за один переход и из C в B можно перейти за один переход.

Вашей программе на вход подаются две строки, содержащие url двух документов A и B.
Выведите Yes, если из A в B можно перейти за два перехода, иначе выведите No.

In [ ]:
https://www.kinopoisk.ru/lists/top250/
https://www.kinopoisk.ru/name/7987/

In [3]:
def url_tester():
#     A = input('Введите ссылку первого документа: ')
#     B = input('Введите ссылку второго документа: ')
    
    A = 'https://www.kinopoisk.ru/lists/top250/'
    B = 'https://www.kinopoisk.ru/name/7987/'
    
    A_url = urlopen(A).read().decode('utf-8') #качаем первую страничку
    A_html = str(A_url) # Делаем из неё строку
    A_soup = BeautifulSoup(A_html, 'html.parser') #варим суп
    A_origin_url = re.search(r'(.+[.]ru|.+[.]com)', A).group() #вытаскиваем оригинальный url сайта для дальнейшей подставки

    B_url = urlopen(B).read().decode('utf-8')
    B_html = str(A_url)
    B_soup = BeautifulSoup(B_html, 'html.parser')


    A_refs = []
    for refs in A_soup.find_all('a', href=True): #проходимся по всем ссылкам первой странички
        url = refs['href']
        print(url)
        if url.startswith('/'): #если ссылка относительная 
            url = A_origin_url + url #то соединяем её с оригинальным url сайта
        if url.startswith(A_origin_url): #проверка на то что ссылка не ввёдет на другой сайт
            if url == B:
                return True
            A_refs.append(url)
    
    
    print(A_refs)


    for refs in A_refs: #проходимся по ссылкам первой страницы и варим из них супы
        C_url = urlopen(refs).read().decode('utf-8')
        C_html = str(C_url)
        C_soup = BeautifulSoup(C_html, 'html.parser')
        
        for refs in C_soup.find_all('a', href=True):
            url = refs['href']
            if url.startswith('/'):
                url = A_origin_url + url
            if url == B:
                return True
    
    return False

url_tester()

https://www.kinopoisk.ru
https://yandex.ru/support/captcha/
['https://www.kinopoisk.ru']


False

Вашей программе на вход подается ссылка на HTML файл.
Вам необходимо скачать этот файл, затем найти в нем все ссылки вида <a ... href="..." ... > и вывести список сайтов, на которые есть ссылка.

In [6]:
def url_tester():
    A = input('Введите ссылку на HTML-файл: ')
   
    A_url = urlopen(A).read().decode('utf-8')
    A_html = str(A_url)
    A_soup = BeautifulSoup(A_html, 'html.parser')
    
    A_origin_url = re.search(r'([\w.]+[.]ru|[\w.]+[.]com)', A).group()
    print(A_origin_url)
    
    A_refs = set()
    for refs in A_soup.find_all('a', href=True):
        if '../' in refs['href']:
            continue 
        A_origin_url = re.search(r'([\w.]+[.]ru|[\w.]+[.]com)', refs['href'])
        if A_origin_url:
            A_origin_url = A_origin_url.group()
            A_refs.add(A_origin_url)
    print(A_refs)
    print(len(A_refs))
url_tester()

Введите ссылку на HTML-файл: https://www.plerdy.com/ru/blog/top-50-web-design-inspiration-websites/
www.plerdy.com
{'ux.stackexchange.com', 'www.uplabs.com', 'www.calltoidea.com', 'www.facebook.com', 'www.thebestdesigns.com', 'www.templatemonster.com', 'www.mockplus.com', 'medium.com', 'alistapart.com', 'www.plerdy.com', 'dribbble.com', 'www.pinterest.com', 'www.uxdesignedge.com', 'www.creativebloq.com', 'www.pentagram.com', 'a.plerdy.com', 'play.google.com', 'www.flickr.com', 'book.com', 'twitter.com', 'uimovement.com', 'itunes.apple.com', 'abduzeedo.com', 'www.youtube.com', 'thegreatdiscontent.com', 'www.linkedin.com', 'uxmag.com', 'www.siteinspire.com', 'www.awwwards.com', 'onepagelove.com', 'collectui.com', 'cssnectar.com', 'pttrns.com', '99designs.com', 'www.smashingmagazine.com'}
35


In [ ]:
http://pastebin.com/raw/7543p0ns
https://www.plerdy.com/ru/blog/top-50-web-design-inspiration-websites/